<a href="https://colab.research.google.com/github/dtoralg/intro_unsupervised/blob/main/notebooks/E2_Clustering_Jeraquico_con_IRIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>